# Memory Management in Python

---

## Table of Contents
1. How Python Manages Memory
2. Reference Counting
3. Garbage Collection
4. The gc Module
5. Memory Optimization with __slots__
6. Weak References
7. Memory Profiling
8. Common Memory Issues
9. Best Practices
10. Key Points
11. Practice Exercises

---

## 1. How Python Manages Memory

Python handles memory automatically through:

1. **Private Heap**: All Python objects are stored in a private heap
2. **Memory Manager**: Handles allocation and deallocation
3. **Reference Counting**: Primary mechanism for memory management
4. **Garbage Collector**: Handles circular references

**Memory architecture:**
```
Object-specific allocators (list, dict, etc.)
           |
Python Object Allocator
           |
Python Raw Memory Allocator (PyMem_*)
           |
System Allocator (malloc/free)
```

In [ ]:
import sys

# Every object has a size in memory
print(f"Empty list: {sys.getsizeof([])} bytes")
print(f"List with 10 ints: {sys.getsizeof(list(range(10)))} bytes")
print(f"Empty dict: {sys.getsizeof({})} bytes")
print(f"Empty string: {sys.getsizeof('')} bytes")
print(f"String 'hello': {sys.getsizeof('hello')} bytes")
print(f"Integer 42: {sys.getsizeof(42)} bytes")
print(f"Float 3.14: {sys.getsizeof(3.14)} bytes")

In [ ]:
# id() returns the memory address of an object
x = [1, 2, 3]
y = x  # Same object
z = [1, 2, 3]  # Different object, same content

print(f"id(x) = {id(x)}")
print(f"id(y) = {id(y)}")
print(f"id(z) = {id(z)}")
print(f"x is y: {x is y}")
print(f"x is z: {x is z}")
print(f"x == z: {x == z}")

In [ ]:
# Small integer caching (-5 to 256)
a = 100
b = 100
print(f"100: a is b = {a is b}")  # True - cached

c = 1000
d = 1000
print(f"1000: c is d = {c is d}")  # Might be False - not cached

# String interning
s1 = "hello"
s2 = "hello"
print(f"'hello': s1 is s2 = {s1 is s2}")  # True - interned

---

## 2. Reference Counting

Every Python object has a reference count that tracks how many references point to it.

In [ ]:
import sys

# sys.getrefcount() returns the reference count
# Note: getrefcount() itself creates a temporary reference (+1)

x = []  # New list, refcount = 1
print(f"After creation: {sys.getrefcount(x) - 1}")  # Subtract 1 for getrefcount

y = x   # Another reference, refcount = 2
print(f"After y = x: {sys.getrefcount(x) - 1}")

z = [x, x, x]  # Three more references
print(f"After z = [x, x, x]: {sys.getrefcount(x) - 1}")

del y  # Remove one reference
print(f"After del y: {sys.getrefcount(x) - 1}")

z.clear()  # Remove references from list
print(f"After z.clear(): {sys.getrefcount(x) - 1}")

In [ ]:
# Reference count changes
class TrackedObject:
    def __init__(self, name):
        self.name = name
        print(f"{name}: Created")
    
    def __del__(self):
        print(f"{self.name}: Destroyed")

print("Creating object...")
obj = TrackedObject("A")

print("\nAdding to list...")
my_list = [obj]

print("\nDeleting original reference...")
del obj

print("\nObject still exists (in list)")

print("\nClearing list...")
my_list.clear()  # Now object is destroyed

In [ ]:
# Circular references - reference counting can't handle these
class Node:
    def __init__(self, name):
        self.name = name
        self.next = None
    
    def __del__(self):
        print(f"Node {self.name}: Destroyed")

# Create circular reference
a = Node("A")
b = Node("B")
a.next = b
b.next = a  # Circular!

print(f"a refcount: {sys.getrefcount(a) - 1}")
print(f"b refcount: {sys.getrefcount(b) - 1}")

# Delete references
del a
del b
print("References deleted, but objects may still exist (circular reference)")

# Force garbage collection
import gc
gc.collect()
print("Garbage collected")

---

## 3. Garbage Collection

Python's garbage collector handles circular references using **generational garbage collection**.

In [ ]:
import gc

# Generational GC: 3 generations (0, 1, 2)
# New objects start in generation 0
# Objects that survive collections are promoted

print(f"GC enabled: {gc.isenabled()}")
print(f"GC thresholds: {gc.get_threshold()}")
# (threshold0, threshold1, threshold2)
# Collection of generation N happens when:
# - allocations - deallocations > thresholdN

In [ ]:
# GC counts
print(f"Collection counts: {gc.get_count()}")
# (count0, count1, count2) - allocations since last collection of each generation

In [ ]:
# Manual garbage collection
class Circular:
    def __init__(self, name):
        self.name = name
        self.ref = None
    
    def __repr__(self):
        return f"Circular({self.name})"

# Create circular references
def create_cycle():
    a = Circular("A")
    b = Circular("B")
    a.ref = b
    b.ref = a
    # Local variables go out of scope
    # But objects have circular reference

# Disable automatic GC temporarily
gc.disable()

create_cycle()
create_cycle()

print(f"Unreachable objects before collect: {len(gc.garbage)}")

# Manual collection
collected = gc.collect()
print(f"Objects collected: {collected}")

gc.enable()

In [ ]:
# Debugging with GC
gc.set_debug(gc.DEBUG_STATS)  # Print collection statistics

# Create some garbage
for _ in range(100):
    create_cycle()

# Force full collection
gc.collect()

gc.set_debug(0)  # Turn off debug

---

## 4. The gc Module

In [ ]:
import gc

# Common gc functions
print("gc module functions:")
print(f"  gc.enable()  - Enable automatic GC")
print(f"  gc.disable() - Disable automatic GC")
print(f"  gc.isenabled() - Check if GC is enabled: {gc.isenabled()}")
print(f"  gc.collect() - Run garbage collection")
print(f"  gc.get_count() - Get allocation counts: {gc.get_count()}")
print(f"  gc.get_threshold() - Get thresholds: {gc.get_threshold()}")
print(f"  gc.set_threshold() - Set thresholds")
print(f"  gc.get_objects() - Get all tracked objects")

In [ ]:
# Get all objects tracked by GC
all_objects = gc.get_objects()
print(f"Total tracked objects: {len(all_objects)}")

# Count by type
from collections import Counter
type_counts = Counter(type(obj).__name__ for obj in all_objects)
print("\nTop 10 object types:")
for type_name, count in type_counts.most_common(10):
    print(f"  {type_name}: {count}")

In [ ]:
# Find references to an object
my_list = [1, 2, 3]

# Get referrers (objects that reference my_list)
referrers = gc.get_referrers(my_list)
print(f"Number of referrers: {len(referrers)}")

# Get referents (objects that my_list references)
referents = gc.get_referents(my_list)
print(f"Referents: {referents}")

In [ ]:
# Custom threshold tuning
# Default: (700, 10, 10)
# - Gen 0 collected every 700 allocations
# - Gen 1 collected every 10 Gen 0 collections
# - Gen 2 collected every 10 Gen 1 collections

original = gc.get_threshold()
print(f"Original threshold: {original}")

# Set more aggressive collection (for memory-constrained apps)
gc.set_threshold(500, 5, 5)
print(f"New threshold: {gc.get_threshold()}")

# Restore
gc.set_threshold(*original)

---

## 5. Memory Optimization with __slots__

In [ ]:
# Regular class uses __dict__ for attributes
class RegularPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Class with __slots__ - no __dict__
class SlottedPoint:
    __slots__ = ['x', 'y']
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Compare memory usage
regular = RegularPoint(1, 2)
slotted = SlottedPoint(1, 2)

print(f"Regular Point size: {sys.getsizeof(regular)} bytes")
print(f"Regular Point __dict__ size: {sys.getsizeof(regular.__dict__)} bytes")
print(f"Slotted Point size: {sys.getsizeof(slotted)} bytes")

try:
    print(slotted.__dict__)
except AttributeError:
    print("Slotted Point has no __dict__")

In [ ]:
# Memory savings with many objects
import tracemalloc

tracemalloc.start()

# Create many regular objects
regular_points = [RegularPoint(i, i) for i in range(10000)]

snapshot1 = tracemalloc.take_snapshot()
regular_size = sum(stat.size for stat in snapshot1.statistics('lineno'))

del regular_points
gc.collect()

# Create many slotted objects
slotted_points = [SlottedPoint(i, i) for i in range(10000)]

snapshot2 = tracemalloc.take_snapshot()
slotted_size = sum(stat.size for stat in snapshot2.statistics('lineno'))

tracemalloc.stop()

print(f"Memory for 10000 regular points: ~{regular_size / 1024:.1f} KB")
print(f"Memory for 10000 slotted points: ~{slotted_size / 1024:.1f} KB")

In [ ]:
# __slots__ restrictions

class SlottedClass:
    __slots__ = ['x', 'y']
    
    def __init__(self):
        self.x = 10
        self.y = 20

obj = SlottedClass()

# Can't add new attributes
try:
    obj.z = 30
except AttributeError as e:
    print(f"Can't add attribute: {e}")

# Defined attributes work fine
print(f"x = {obj.x}, y = {obj.y}")

In [ ]:
# __slots__ with inheritance

class Base:
    __slots__ = ['a']

class Derived(Base):
    __slots__ = ['b']  # Add more slots

d = Derived()
d.a = 1  # From Base
d.b = 2  # From Derived
print(f"a = {d.a}, b = {d.b}")

# If derived class doesn't define __slots__, it gets __dict__
class DerivedNoSlots(Base):
    pass

d2 = DerivedNoSlots()
d2.a = 1
d2.c = 3  # Can add dynamic attributes
print(f"DerivedNoSlots has __dict__: {hasattr(d2, '__dict__')}")

In [ ]:
# __slots__ with __dict__ (allow both)

class Hybrid:
    __slots__ = ['x', 'y', '__dict__']  # Include __dict__
    
    def __init__(self):
        self.x = 1  # Goes to slot
        self.y = 2  # Goes to slot
        self.z = 3  # Goes to __dict__

h = Hybrid()
print(f"Slots: x={h.x}, y={h.y}")
print(f"Dict attributes: {h.__dict__}")

---

## 6. Weak References

Weak references allow you to reference objects without preventing garbage collection.

In [ ]:
import weakref

class ExpensiveObject:
    def __init__(self, name):
        self.name = name
        print(f"{name}: Created")
    
    def __del__(self):
        print(f"{self.name}: Destroyed")

# Create object and weak reference
obj = ExpensiveObject("Heavy")
weak_ref = weakref.ref(obj)

print(f"Object: {weak_ref()}")
print(f"Object name: {weak_ref().name}")

# Delete strong reference
del obj
gc.collect()

# Weak reference now returns None
print(f"After deletion: {weak_ref()}")

In [ ]:
# Weak reference callback
def callback(ref):
    print(f"Object was garbage collected! Reference: {ref}")

obj = ExpensiveObject("Tracked")
weak_ref = weakref.ref(obj, callback)

print("Deleting object...")
del obj
gc.collect()

In [ ]:
# WeakValueDictionary - values are weak references
class Cache:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    
    def get(self, key):
        return self._cache.get(key)
    
    def set(self, key, value):
        self._cache[key] = value
    
    def __len__(self):
        return len(self._cache)

cache = Cache()

# Create and cache an object
obj = ExpensiveObject("Cached")
cache.set("key1", obj)

print(f"Cache size: {len(cache)}")
print(f"Retrieved: {cache.get('key1').name}")

# Delete the original reference
del obj
gc.collect()

# Object is gone from cache
print(f"Cache size after del: {len(cache)}")
print(f"Retrieved after del: {cache.get('key1')}")

In [ ]:
# WeakKeyDictionary - keys are weak references
# Useful for associating data with objects without preventing GC

metadata = weakref.WeakKeyDictionary()

class User:
    def __init__(self, name):
        self.name = name

user1 = User("Alice")
user2 = User("Bob")

# Store metadata
metadata[user1] = {"login_count": 5}
metadata[user2] = {"login_count": 3}

print(f"Metadata entries: {len(metadata)}")

# Delete a user
del user1
gc.collect()

print(f"Metadata entries after del: {len(metadata)}")

In [ ]:
# WeakSet
observers = weakref.WeakSet()

class Observer:
    def __init__(self, name):
        self.name = name
    
    def notify(self, msg):
        print(f"{self.name} received: {msg}")

# Create observers
obs1 = Observer("Observer1")
obs2 = Observer("Observer2")

observers.add(obs1)
observers.add(obs2)

print(f"Observers: {len(observers)}")

# Notify all
for obs in observers:
    obs.notify("Hello!")

# Delete one observer
del obs1
gc.collect()

print(f"Observers after del: {len(observers)}")

In [ ]:
# finalize - more robust than __del__
def cleanup(name):
    print(f"Cleaning up {name}")

class Resource:
    def __init__(self, name):
        self.name = name
        self._finalizer = weakref.finalize(self, cleanup, name)
    
    def close(self):
        """Explicitly close resource."""
        self._finalizer()
    
    @property
    def alive(self):
        return self._finalizer.alive

r = Resource("File")
print(f"Resource alive: {r.alive}")

r.close()  # Explicit cleanup
print(f"Resource alive after close: {r.alive}")

# Won't run cleanup again when deleted
del r

---

## 7. Memory Profiling

In [ ]:
# tracemalloc - built-in memory profiler
import tracemalloc

tracemalloc.start()

# Do some work
data = [list(range(1000)) for _ in range(100)]

snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("Top 5 memory allocations:")
for stat in top_stats[:5]:
    print(stat)

tracemalloc.stop()

In [ ]:
# Compare memory snapshots
tracemalloc.start()

snapshot1 = tracemalloc.take_snapshot()

# Allocate more memory
large_list = [i ** 2 for i in range(100000)]

snapshot2 = tracemalloc.take_snapshot()

# Compare
top_stats = snapshot2.compare_to(snapshot1, 'lineno')

print("Memory difference:")
for stat in top_stats[:5]:
    print(stat)

tracemalloc.stop()

In [ ]:
# Memory usage tracker function
def memory_usage():
    """Return current memory usage in MB."""
    import tracemalloc
    current, peak = tracemalloc.get_traced_memory()
    return current / 1024 / 1024, peak / 1024 / 1024

tracemalloc.start()

current, peak = memory_usage()
print(f"Initial: {current:.2f} MB, Peak: {peak:.2f} MB")

# Allocate memory
data = ["x" * 1000 for _ in range(10000)]

current, peak = memory_usage()
print(f"After allocation: {current:.2f} MB, Peak: {peak:.2f} MB")

# Free memory
del data
gc.collect()

current, peak = memory_usage()
print(f"After deletion: {current:.2f} MB, Peak: {peak:.2f} MB")

tracemalloc.stop()

In [ ]:
# Context manager for memory tracking
from contextlib import contextmanager

@contextmanager
def track_memory():
    tracemalloc.start()
    yield
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"Memory used: {current / 1024:.2f} KB")
    print(f"Peak memory: {peak / 1024:.2f} KB")

# Usage
with track_memory():
    data = [i for i in range(100000)]

---

## 8. Common Memory Issues

In [ ]:
# Issue 1: Unintended references in closures
def create_functions():
    functions = []
    for i in range(5):
        # This creates a closure that references 'i'
        def func():
            return i  # All functions reference the same 'i'
        functions.append(func)
    return functions

funcs = create_functions()
print([f() for f in funcs])  # All return 4!

# Fix: Use default argument
def create_functions_fixed():
    functions = []
    for i in range(5):
        def func(x=i):  # Capture current value
            return x
        functions.append(func)
    return functions

funcs_fixed = create_functions_fixed()
print([f() for f in funcs_fixed])  # Correct!

In [ ]:
# Issue 2: Large default arguments
def bad_function(items=[]):  # Mutable default!
    items.append(1)
    return items

print(bad_function())  # [1]
print(bad_function())  # [1, 1] - Same list!
print(bad_function())  # [1, 1, 1]

# Fix: Use None as default
def good_function(items=None):
    if items is None:
        items = []
    items.append(1)
    return items

print(good_function())  # [1]
print(good_function())  # [1] - New list each time

In [ ]:
# Issue 3: Circular references with __del__
# Classes with __del__ and circular references can't be collected

class BadCircular:
    def __init__(self):
        self.ref = None
    
    def __del__(self):
        # This prevents collection of circular references
        print("Destroyed")

# Fix: Use weakref
class GoodCircular:
    def __init__(self):
        self._ref = None
    
    @property
    def ref(self):
        return self._ref() if self._ref else None
    
    @ref.setter
    def ref(self, value):
        self._ref = weakref.ref(value) if value else None

a = GoodCircular()
b = GoodCircular()
a.ref = b
b.ref = a  # No memory leak!

In [ ]:
# Issue 4: Growing caches without bounds
def memoize(func):
    cache = {}  # Grows forever!
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrapper

# Fix: Use LRU cache with max size
from functools import lru_cache

@lru_cache(maxsize=128)  # Limited size
def good_cached_function(n):
    return n ** 2

# Or use custom bounded cache
from collections import OrderedDict

class LRUCache:
    def __init__(self, maxsize=100):
        self.cache = OrderedDict()
        self.maxsize = maxsize
    
    def get(self, key):
        if key in self.cache:
            self.cache.move_to_end(key)
            return self.cache[key]
        return None
    
    def set(self, key, value):
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.maxsize:
            self.cache.popitem(last=False)

In [ ]:
# Issue 5: Large string concatenation
import time

# Bad: Creates many intermediate strings
def bad_concat(n):
    result = ""
    for i in range(n):
        result += str(i)  # Creates new string each time
    return result

# Good: Use join
def good_concat(n):
    return "".join(str(i) for i in range(n))

# Compare
n = 10000

start = time.time()
bad_concat(n)
print(f"Bad concat: {time.time() - start:.4f}s")

start = time.time()
good_concat(n)
print(f"Good concat: {time.time() - start:.4f}s")

---

## 9. Best Practices

In [ ]:
# Best practices summary
best_practices = """
Memory Management Best Practices:

1. Use generators for large sequences
   - list(range(1000000))  # Bad - stores all in memory
   - (x for x in range(1000000))  # Good - lazy iteration

2. Use __slots__ for classes with many instances
   - Saves ~50% memory per instance
   - Faster attribute access

3. Avoid circular references or use weakref
   - Circular refs need GC to clean up
   - weakref allows references without preventing collection

4. Close resources explicitly (context managers)
   - with open() as f: ...
   - Don't rely on __del__ for cleanup

5. Use bounded caches (lru_cache with maxsize)
   - Prevents unbounded memory growth

6. Profile before optimizing
   - Use tracemalloc to find actual problems
   - Don't optimize prematurely

7. Prefer built-in types when possible
   - Highly optimized in C
   - Use array.array for homogeneous numeric data

8. Delete large objects when done
   - del large_object
   - gc.collect() if needed immediately

9. Use appropriate data structures
   - set for membership testing (O(1)) vs list (O(n))
   - deque for queue operations

10. Consider memory-mapped files for very large data
    - mmap module for files larger than RAM
"""
print(best_practices)

In [ ]:
# Example: Generator vs List
import tracemalloc

def sum_with_list(n):
    return sum([i ** 2 for i in range(n)])

def sum_with_generator(n):
    return sum(i ** 2 for i in range(n))

n = 100000

# With list
tracemalloc.start()
sum_with_list(n)
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"List comprehension - Peak: {peak / 1024:.2f} KB")

# With generator
tracemalloc.start()
sum_with_generator(n)
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Generator expression - Peak: {peak / 1024:.2f} KB")

---

## 10. Key Points

1. **Reference Counting**: Primary mechanism - objects deleted when count reaches 0
2. **Garbage Collector**: Handles circular references using generational GC
3. **gc module**: Manual control over garbage collection
4. **__slots__**: Reduce memory by eliminating __dict__
5. **Weak References**: Reference objects without preventing GC
6. **tracemalloc**: Built-in memory profiling
7. **Common Issues**: Circular refs, growing caches, closure refs
8. **Generators**: Memory-efficient iteration
9. **Context Managers**: Proper resource cleanup
10. **Profile First**: Use tools to find actual memory problems

---

## 11. Practice Exercises

In [ ]:
# Exercise 1: Memory-efficient data class
# Create a Point class using __slots__ that:
# - Stores x, y, z coordinates
# - Has a method to calculate distance from origin
# - Compare memory usage with regular class

class Point:
    pass

# Test:
# points = [Point(i, i+1, i+2) for i in range(10000)]
# Measure memory usage

In [ ]:
# Exercise 2: Observer pattern with weak references
# Implement a Subject class that:
# - Can have observers added/removed
# - Uses weak references so observers can be GC'd
# - Notifies all living observers on change

class Subject:
    pass

class Observer:
    pass

# Test:
# subject = Subject()
# obs = Observer()
# subject.add_observer(obs)
# subject.notify("test")
# del obs
# subject.notify("test2")  # Should not crash

In [ ]:
# Exercise 3: Bounded cache decorator
# Create a decorator that:
# - Caches function results
# - Has a maximum size limit
# - Evicts oldest entries when full

def bounded_cache(maxsize=100):
    pass

# Test:
# @bounded_cache(maxsize=3)
# def expensive(x):
#     return x ** 2
#
# expensive(1)  # Computed
# expensive(2)  # Computed
# expensive(3)  # Computed
# expensive(1)  # From cache
# expensive(4)  # Computed, evicts oldest

In [ ]:
# Exercise 4: Memory leak detector
# Create a context manager that:
# - Tracks all objects created inside it
# - Reports objects not cleaned up after exit
# - Shows potential memory leaks

class MemoryLeakDetector:
    pass

# Test:
# with MemoryLeakDetector() as detector:
#     x = [1, 2, 3]
#     y = {"a": 1}
# # Should report no leaks if x, y go out of scope

In [ ]:
# Exercise 5: Object pool
# Implement an object pool that:
# - Reuses objects instead of creating new ones
# - Has a maximum pool size
# - Tracks created/reused statistics

class ObjectPool:
    pass

# Test:
# pool = ObjectPool(create_func=lambda: [], max_size=5)
# obj1 = pool.acquire()
# obj1.append(1)
# pool.release(obj1)
# obj2 = pool.acquire()  # Should reuse obj1
# print(pool.stats())  # {created: 1, reused: 1}

---

## Solutions

In [ ]:
# Solution 1:
import math

class PointRegular:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    
    def distance_from_origin(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)

class PointSlotted:
    __slots__ = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    
    def distance_from_origin(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)

# Compare
p1 = PointRegular(3, 4, 5)
p2 = PointSlotted(3, 4, 5)

print(f"Regular size: {sys.getsizeof(p1)} + dict: {sys.getsizeof(p1.__dict__)} bytes")
print(f"Slotted size: {sys.getsizeof(p2)} bytes")
print(f"Distance: {p2.distance_from_origin():.2f}")

In [ ]:
# Solution 2:
import weakref

class Subject:
    def __init__(self):
        self._observers = weakref.WeakSet()
    
    def add_observer(self, observer):
        self._observers.add(observer)
    
    def remove_observer(self, observer):
        self._observers.discard(observer)
    
    def notify(self, message):
        for observer in self._observers:
            observer.update(message)

class Observer:
    def __init__(self, name):
        self.name = name
    
    def update(self, message):
        print(f"{self.name} received: {message}")

# Test
subject = Subject()
obs1 = Observer("Observer1")
obs2 = Observer("Observer2")

subject.add_observer(obs1)
subject.add_observer(obs2)

print("Notifying all:")
subject.notify("Hello!")

print("\nDeleting obs1:")
del obs1
gc.collect()

print("\nNotifying remaining:")
subject.notify("Still here?")

In [ ]:
# Solution 3:
from collections import OrderedDict
from functools import wraps

def bounded_cache(maxsize=100):
    def decorator(func):
        cache = OrderedDict()
        hits = 0
        misses = 0
        
        @wraps(func)
        def wrapper(*args):
            nonlocal hits, misses
            if args in cache:
                cache.move_to_end(args)
                hits += 1
                return cache[args]
            
            result = func(*args)
            cache[args] = result
            misses += 1
            
            if len(cache) > maxsize:
                cache.popitem(last=False)
            
            return result
        
        def cache_info():
            return {"hits": hits, "misses": misses, "size": len(cache)}
        
        wrapper.cache_info = cache_info
        return wrapper
    return decorator

@bounded_cache(maxsize=3)
def expensive(x):
    print(f"Computing {x}...")
    return x ** 2

expensive(1)
expensive(2)
expensive(3)
expensive(1)  # From cache
expensive(4)  # Evicts 2
expensive(2)  # Recomputes

print(f"Cache info: {expensive.cache_info()}")

In [ ]:
# Solution 4:
class MemoryLeakDetector:
    def __init__(self):
        self.before = None
    
    def __enter__(self):
        gc.collect()
        self.before = set(id(obj) for obj in gc.get_objects())
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        gc.collect()
        after = set(id(obj) for obj in gc.get_objects())
        new_objects = after - self.before
        
        if new_objects:
            print(f"\nPotential leaks: {len(new_objects)} new objects")
            
            # Find types of new objects
            type_counts = {}
            for obj in gc.get_objects():
                if id(obj) in new_objects:
                    t = type(obj).__name__
                    type_counts[t] = type_counts.get(t, 0) + 1
            
            print("Types of new objects:")
            for t, count in sorted(type_counts.items(), key=lambda x: -x[1])[:5]:
                print(f"  {t}: {count}")
        else:
            print("\nNo memory leaks detected!")
        
        return False

# Test
with MemoryLeakDetector():
    temp = [1, 2, 3]  # Will be cleaned up

print("---")

# Test with leak
leaked = []
with MemoryLeakDetector():
    leaked.append([1, 2, 3])  # This survives

In [ ]:
# Solution 5:
class ObjectPool:
    def __init__(self, create_func, max_size=10, reset_func=None):
        self.create_func = create_func
        self.reset_func = reset_func or (lambda x: x.clear() if hasattr(x, 'clear') else None)
        self.max_size = max_size
        self.pool = []
        self.created = 0
        self.reused = 0
    
    def acquire(self):
        if self.pool:
            obj = self.pool.pop()
            self.reused += 1
            return obj
        else:
            self.created += 1
            return self.create_func()
    
    def release(self, obj):
        if len(self.pool) < self.max_size:
            self.reset_func(obj)  # Reset object state
            self.pool.append(obj)
    
    def stats(self):
        return {
            "created": self.created,
            "reused": self.reused,
            "pool_size": len(self.pool)
        }

# Test
pool = ObjectPool(create_func=list, max_size=3)

obj1 = pool.acquire()
obj1.extend([1, 2, 3])
print(f"obj1: {obj1}")

pool.release(obj1)

obj2 = pool.acquire()  # Reuses obj1
print(f"obj2 (reused, reset): {obj2}")

print(f"Stats: {pool.stats()}")